<a href="https://colab.research.google.com/github/szhou12/ml-practice/blob/main/fill_in_mask_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fill-In-Mask Task

Resource:
[A Sentence Fill-in-The-Blank Example Using Hugging Face](https://jamesmccaffrey.wordpress.com/2021/10/05/a-sentence-fill-in-the-blank-example-using-hugging-face/)

In [1]:
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 5.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import datasets
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import numpy as np

In [ ]:
# from transformers import pipeline

In [ ]:
raw_dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(raw_dataset["train"])

Dataset({
    features: ['text'],
    num_rows: 36718
})


In [ ]:
raw_dataset["train"]['text'][10]

' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a charac

In [ ]:
checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_input = raw_dataset["train"]['text'][10]

# Ref: https://stackoverflow.com/questions/66625389/attributeerror-list-object-has-no-attribute-size-hugging-face-transformers
encoded_input = tokenizer(raw_input, return_tensors='pt') # must return pytorch tensor in order for model to run
encoded_input

{'input_ids': tensor([[  101,  1996,  2208,  1005,  1055,  2645,  2291,  1010,  1996, 22312,
          2291,  1010,  2003,  3344,  2058,  3495,  2013, 11748,  4801,  7895,
         11906,  1012,  2076,  6416,  1010,  2867,  7276,  2169,  3131,  2478,
          1037,  2327,  1030,  1011,  1030,  2091,  7339,  1997,  1996, 11686,
          4949,  1024,  2320,  1037,  2839,  2003,  3479,  1010,  1996,  2447,
          5829,  1996,  2839,  2105,  1996, 11686,  1999,  2353,  1030,  1011,
          1030,  2711,  1012,  1037,  2839,  2064,  2069,  2552,  2320,  2566,
          1030,  1011,  1030,  2735,  1010,  2021,  3494,  2064,  2022,  4379,
          3674,  4332,  2012,  1996, 10961,  1997,  2060,  3494,  1005,  4332,
          1012,  2169,  2839,  2038,  1037,  2492,  1998,  3292,  1997,  2929,
          3132,  2011,  2037,  2895,  7633,  1012,  2039,  2000,  3157,  3494,
          2064,  2022,  4137,  2000,  1037,  2309,  3260,  1012,  2076, 11247,
          1010,  3494,  2097,  2655,  

In [ ]:
## check the word that is gonna be masked
word = tokenizer(raw_input, return_tensors='pt')['input_ids'][0][5] # ID = 2645
tokenizer.decode(word)

'battle'

In [ ]:
# masking idx=5 token
# print(tokenizer.mask_token_id)
encoded_input['input_ids'][0][5] = tokenizer.mask_token_id
encoded_input

{'input_ids': tensor([[  101,  1996,  2208,  1005,  1055,   103,  2291,  1010,  1996, 22312,
          2291,  1010,  2003,  3344,  2058,  3495,  2013, 11748,  4801,  7895,
         11906,  1012,  2076,  6416,  1010,  2867,  7276,  2169,  3131,  2478,
          1037,  2327,  1030,  1011,  1030,  2091,  7339,  1997,  1996, 11686,
          4949,  1024,  2320,  1037,  2839,  2003,  3479,  1010,  1996,  2447,
          5829,  1996,  2839,  2105,  1996, 11686,  1999,  2353,  1030,  1011,
          1030,  2711,  1012,  1037,  2839,  2064,  2069,  2552,  2320,  2566,
          1030,  1011,  1030,  2735,  1010,  2021,  3494,  2064,  2022,  4379,
          3674,  4332,  2012,  1996, 10961,  1997,  2060,  3494,  1005,  4332,
          1012,  2169,  2839,  2038,  1037,  2492,  1998,  3292,  1997,  2929,
          3132,  2011,  2037,  2895,  7633,  1012,  2039,  2000,  3157,  3494,
          2064,  2022,  4137,  2000,  1037,  2309,  3260,  1012,  2076, 11247,
          1010,  3494,  2097,  2655,  

In [ ]:
blank_id_idx = torch.where(encoded_input["input_ids"] == tokenizer.mask_token_id)[1]

model = AutoModelForMaskedLM.from_pretrained(checkpoint)

with torch.no_grad(): # ?
    all_logits = model(**encoded_input).logits

# torch.Size([1, 323, 30522]): 323 = total # tokens in encoded_input; 30522 = total # logits for each token
# all_logits.shape

In [ ]:
# get all predicted logits for the masked token (idx=5)
# torch.Size([1, 30522])
pred_logits = all_logits[0, blank_id_idx, :]

In [ ]:
torch.topk(pred_logits, 5, dim=1)

torch.return_types.topk(
values=tensor([[10.2755,  8.7071,  8.5609,  8.4958,  8.2816]]),
indices=tensor([[ 4337, 17762,  5464,  2645,  8608]]))

In [ ]:
# torch.topk(pred_logits, 5, dim=1).indices gives tensor = (1 x 5)
torch.topk(pred_logits, 5, dim=1).indices[0]

tensor([ 4337, 17762,  5464,  2645,  8608])

In [ ]:
top_ids = torch.topk(pred_logits, 5, dim=1).indices[0].tolist()
print(top_ids)

[4337, 17762, 5464, 2645, 8608]


## The top five predicteds as words: 

In [ ]:
for id in top_ids:
  print(tokenizer.decode([id]))

combat
multiplayer
ranking
battle
tactical


## Converting raw logit outputs to probabilities

In [ ]:
np.set_printoptions(precision=4, suppress=True)
pred_probs = torch.nn.functional.softmax(pred_logits, dim=-1).numpy()
pred_probs

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
np.sort(pred_probs[0])

array([0.    , 0.    , 0.    , ..., 0.0348, 0.0403, 0.1932], dtype=float32)

Python Slice notation: `list[<start>:<stop>:<step>]`

`a[::-1]`: reverse list a

In [ ]:
pred_probs = np.sort(pred_probs[0])[::-1] # sort in ascending order and reverse

In [ ]:
top_probs = pred_probs[:5]
top_probs

array([0.1932, 0.0403, 0.0348, 0.0326, 0.0263], dtype=float32)